In [1]:
import pandas as pd
import numpy as np
import random 
import os
import joblib
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import log_loss, roc_auc_score, recall_score, precision_score, average_precision_score, f1_score, classification_report, accuracy_score,confusion_matrix, silhouette_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict

from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.datasets import make_blobs

import warnings
warnings.filterwarnings("ignore")
os.chdir("/Users/yj.noh/Documents/GitHub")
print(os.getcwd())

/Users/yj.noh/Documents/GitHub


In [4]:
data = pd.read_csv("/Users/yj.noh/Desktop/total_working.csv", encoding = "utf-8")
data = data.sort_values(by=["brms_rider_id"])
print(data.head())
print(data.shape)

                                           brms_rider_id  birth  \
18635  0003198AAE6FE85D1F86B2D4D45F992CFAE1D3E22256C0...   1990   
16292  000D03494DB444F1F929C5A73DE30DAC8BDEAB5F6530B1...   1950   
2997   0011233F41929E879E97D4B1CB9C186CACAFBF7DEB3C71...   1990   
17703  00154284CDD04FFA4526ACF5D70A7C229F3B359CFD328A...   1990   
14154  001711D89BF29875A95FB9B9DBAE335B75B0C110F57190...   1960   

      delivery_method first_available_date business_start_day  is_recom  \
18635            BIKE             2023.9.8          2023.9.14         0   
16292            WALK            2023.9.25                NaN         0   
2997             WALK            2023.8.25                NaN         1   
17703            BIKE            2023.9.28          2023.9.29         1   
14154            BIKE            2023.9.11                NaN         0   

               insurance_type gender  \
18635  BIKE_ON_DEMAND_HYUNDAI    MAN   
16292            NOT_REQUIRED  WOMAN   
2997             NOT_REQUI

In [8]:
# 친구추천 데이터 삭제 
print(data['is_recom'].value_counts()) 

df = data[data['is_recom'] == 0 ] 
print(df.shape) #14,250 

is_recom
0    14250
1     6949
Name: count, dtype: int64
(14250, 34)


In [9]:
# 날짜 형식 변환
df['first_available_date'] = pd.to_datetime(df['first_available_date'], format='%Y.%m.%d')
df['business_start_day'] = pd.to_datetime(df['business_start_day'], format='%Y.%m.%d')


## 1. first_available_date 없는 사람 / 첫 배달 가능일 존재, 실제 첫 배달 수행 여부 x 제외 

In [18]:
null_first_available_date_count = df['first_available_date'].isnull().sum()
count_missing_business_start_day = df['first_available_date'].notnull() & df['business_start_day'].isnull()


print(f"'first_available_date'가 null인 데이터 개수: {null_first_available_date_count} 개") # 2104명
print(f"'first_available_date'는 있지만 'business_start_day'가 없는 데이터 개수: {count_missing_business_start_day.sum()} 개") #5818명 

'first_available_date'가 null인 데이터 개수: 2104 개
'first_available_date'는 있지만 'business_start_day'가 없는 데이터 개수: 5818 개
